Librerías y conexión a la base de datos

Se importan las librerías para las bases de datos

In [1]:
import pandas as pd
import numpy as np
#import sqlite3
from openpyxl import Workbook
#from dbi import connect
import pyodbc
#from pyodbc import connect as pyodbc_connect
#from sqlite3 import connect as sqlite3_connect
from scipy import stats
from scipy import special
from scipy.stats import gmean

In [2]:
import funciones

In [3]:
#Almacenamos la información del servidor para poder realizar la conexion más adelante
server = 'ipcprod.database.windows.net'
user = 'ipcreader'
password = '1pc/*2023'
database = 'db-indices'

In [4]:
#Se crea un string para realizar la conexión, con la informacion del servidor almacenada previamente
conn_str = (
    f'Driver={{ODBC Driver 17 for SQL Server}};'
    f'Server={server};'
    f'Database={database};'
    f'Uid={user};'
    f'Pwd={password};'
)

In [5]:
try:
    conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()
    
    cursor.execute("EXEC [dbo].[sp_get_precios_recolectados_mes] 2024, 1")
    boletas = pd.DataFrame([tuple(row) for row in cursor.fetchall()], columns=[column[0] for column in cursor.description])
    
    cursor.execute("EXEC sp_get_indice_grupo 2024, 2")
    ponderaciones = pd.DataFrame([tuple(row) for row in cursor.fetchall()], columns=[column[0] for column in cursor.description])

    conn.close()
    print("Connection successful!")
except Exception as e:
    print(f"Error: {e}")

Connection successful!


In [26]:
#boletas

In [7]:
#esta parte la voy a usar para poder modificar la base de datos, luego la quito de este python
boletas01 = boletas[['region', 'codigo_articulo', 'ine_poll_id', 'cantidad_anterior', 'cantidad_actual', 'precio_anterior', 'precio_actual', 'nt_tipo']]
boletas01.rename(columns={'region':'RegCod', 'codigo_articulo':'ArtCod', 'ine_poll_id':'BolNum', 'cantidad_anterior':'UraChi', 'cantidad_actual':'UreCan', 'precio_anterior':'ArtPhi', 'precio_actual':'ArtPac', 'nt_tipo':'ArtCR'}, inplace=True)
boletas01.set_index(['RegCod', 'ArtCod', 'BolNum'], inplace=True)
boletas01.sort_index(inplace=True)
boletas01

C:\Users\sncap\AppData\Local\Temp\ipykernel_17248\3368766721.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  boletas01.rename(columns={'region':'RegCod', 'codigo_articulo':'ArtCod', 'ine_poll_id':'BolNum', 'cantidad_anterior':'UraChi', 'cantidad_actual':'UreCan', 'precio_anterior':'ArtPhi', 'precio_actual':'ArtPac', 'nt_tipo':'ArtCR'}, inplace=True)
C:\Users\sncap\AppData\Local\Temp\ipykernel_17248\3368766721.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  boletas01.sort_index(inplace=True)


UraChi UreCan   ArtPhi   ArtPac  ArtCR
RegCod ArtCod   BolNum                                             
1      10101011 1073-1-2024    1.00   1.00   300.00   300.00    NaN
                1209-1-2024    1.00   1.00  1873.50  1873.50    NaN
                1676-1-2024    1.00   1.00   750.00   750.00    NaN
                3198-1-2024    1.00   1.00   700.00   700.00    NaN
                3308-1-2024    1.00   1.00  1450.00  1450.00    NaN
...                             ...    ...      ...      ...    ...
8      9740083  16255-1-2024   1.00   1.00     2.00     2.00    NaN
                16326-1-2024   1.00   1.00     2.00     2.00    NaN
                16350-1-2024   1.00   1.00     2.00     2.00    NaN
                16485-1-2024   1.00   1.00     2.00     2.00    NaN
                20848-1-2024   1.00   1.00     2.00     2.00    NaN

[51527 rows x 5 columns]

In [22]:
test = boletas

In [9]:
#test['codigo_articulo'] = test['codigo_articulo'].replace(funciones.recodificacion)
#test.sort_index() #si funciona yay!!!! 

In [10]:
#test[test['codigo_articulo'].str.len()!=9]

In [25]:
#con este codigo practicamente tengo lo que queria
test['codigo_articulo'] = test['codigo_articulo'].replace(funciones.recodificacion) #esta parte hace la recodificacion con el diccionario de los codigos en el archivo funciones
test['decada'] = test['decada'].replace(funciones.decadas)
test['nt_tipo'] = test['nt_tipo'].replace({351761:'S', 351761:'S'})
test['nt_tipo'] = test['nt_tipo'].fillna('N')
test01 = test[['region', 'codigo_articulo', 'ine_poll_id', 'cantidad_anterior', 'cantidad_actual', 'precio_anterior', 'precio_actual', 'nt_tipo', 'decada']]
test01.rename(columns={'region':'RegCod', 'codigo_articulo':'ArtCod', 'ine_poll_id':'BolNum', 'cantidad_anterior':'UraChi', 'cantidad_actual':'UreCan', 'precio_anterior':'ArtPhi', 'precio_actual':'ArtPac', 'nt_tipo':'ArtCR', 'decada':'PerSem'}, inplace=True)
test01 = test01[test01['ArtCod'].str.len() == 9]
test01.set_index(['RegCod', 'ArtCod', 'BolNum'], inplace=True)
test01.sort_index(inplace=True)

test01

C:\Users\sncap\AppData\Local\Temp\ipykernel_17248\2775723068.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test01.rename(columns={'region':'RegCod', 'codigo_articulo':'ArtCod', 'ine_poll_id':'BolNum', 'cantidad_anterior':'UraChi', 'cantidad_actual':'UreCan', 'precio_anterior':'ArtPhi', 'precio_actual':'ArtPac', 'nt_tipo':'ArtCR', 'decada':'PerSem'}, inplace=True)


UraChi  UreCan   ArtPhi   ArtPac ArtCR PerSem
RegCod ArtCod    BolNum                                                     
1      011110101 1035-1-2024   400.00  400.00     7.10     7.10     N      1
                 1104-1-2024   400.00  400.00     7.50     7.50     N      2
                 1115-1-2024   400.00  400.00     7.00     7.00     N      1
                 113-1-2024    400.00  400.00     7.00     7.15     N      1
                 1175-1-2024   400.00  400.00     7.10     7.10     N      2
...                               ...     ...      ...      ...   ...    ...
8      125110401 16025-1-2024    1.00    1.00  2500.00  2500.00     N      1
                 16027-1-2024    1.00    1.00  2000.00  2000.00     N      1
                 16052-1-2024    1.00    1.00  3800.00  3800.00     N      3
                 16435-1-2024    1.00    1.00  2800.00  2800.00     N      1
                 16444-1-2024    1.00    1.00  4000.00  4000.00     N      1

[46352 rows x 6 columns]

___________________________________________________________________________________________

In [12]:
participacion = pd.DataFrame({
        'region': [1, 2, 3, 4, 5, 6, 7, 8],
        'participacion': [13.472203, 12.046949, 12.486967, 12.483411, 12.674423, 12.415842, 12.190962, 12.229243]
    })
participacion['region'] = participacion['region'].astype('int64')

In [13]:
# Creamos un dataframe con las columnas que se van a utilizar
# Selecciona las columnas que se van a utilizar
bol = boletas[boletas['tp_poll_impute_status'].isin([351821, 351824])]
bol = boletas[boletas['detail_tp_poll_status'].isin([351634, 351635])]
bol = boletas[['region', 'codigo_articulo', 'articulo', 'cantidad_anterior', 'cantidad_actual', 'precio_anterior', 'precio_actual', 'nt_tipo']]
bol.insert(1, 'cod_prod', 
               np.where(bol['codigo_articulo'].str.len() > 8, 
                        bol['codigo_articulo'].str.slice(0, -2), 
                        bol['codigo_articulo'].str.slice(0, -1)))
bol.loc[:, 'cod_prod'] = bol['cod_prod'].apply(lambda x: '0' + x if len(x) == 6 else x)
bol['cantidad_anterior'] = bol['cantidad_anterior'].astype(float)
bol['cantidad_actual'] = bol['cantidad_actual'].astype(float)
bol['precio_anterior'] = bol['precio_anterior'].astype(float)
bol['precio_actual'] = bol['precio_actual'].astype(float)
#boletas['cantidad_anterior'] = boletas.apply(lambda row: row['cantidad_actual'] if row['cantidad_anterior'] == 0 or pd.isnull(row['cantidad_anterior']) else row['cantidad_anterior'], axis=1)
#boletas['precio_anterior'] = boletas.apply(lambda row: row['precio_actual'] if row['precio_anterior'] == 0 or pd.isnull(row['precio_anterior']) else row['precio_anterior'], axis=1)

#bol

C:\Users\sncap\AppData\Local\Temp\ipykernel_17248\2388987865.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bol.loc[:, 'cod_prod'] = bol['cod_prod'].apply(lambda x: '0' + x if len(x) == 6 else x)
C:\Users\sncap\AppData\Local\Temp\ipykernel_17248\2388987865.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bol['cantidad_anterior'] = bol['cantidad_anterior'].astype(float)
C:\Users\sncap\AppData\Local\Temp\ipykernel_17248\2388987865.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy

In [14]:
#boletas['cantidad_anterior'].isnull().sum()
#bol['cantidad_actual'].isnull().sum()


#bol['cantidad_anterior'].dtype
#bol['cantidad_actual'].dtype
bol['precio_anterior'].dtype
#bol['precio_actual'].dtype

dtype('float64')

In [15]:
# Se obtienen las ponderaciones de cada produto por cada region y los indices anteriores
funciones.pondcat('Producto', ponderaciones)

,region,grupo_codigo,grupo_nombre,ponderacion_region,indice_anterior
4,0,111101,Arroz,0.522367,100.054381
5,0,111102,Maíz,0.711087,99.958663
7,0,111201,Harina de trigo,0.055905,98.964196
8,0,111202,Harina de maíz,0.170136,100.390846
9,0,111203,Harina para atoles,0.143783,99.951303
...,...,...,...,...,...
6817,8,1390901,Servicios legales y financieros,0.194022,100.000000
6818,8,1390902,Servicios funerarios,0.037005,100.000000
6819,8,1390903,Servicios de registro civil,0.056273,100.000000
6820,8,1390904,Pago de impuestos de circulación de vehículo,0.146902,100.000000


Calculo del IPC

In [16]:
#funciones que luego voy a pasar al archivo que tiene todas las funciones
def variacion(x):
    x['var_prod'] = (x['precio_actual'] / x['cantidad_actual']) * (x['cantidad_anterior'] / x['precio_anterior'])
    return x

In [17]:
# Se calcula la variacion de las variedades
variacion = variacion(bol)
variacion['var_prod'] = bol.apply(lambda row: 1 if row['nt_tipo'] in [351761, 351762] else row['var_prod'], axis=1)
variacion = variacion[['region', 'cod_prod', 'codigo_articulo', 'articulo', 'var_prod']]

#bol['var_prod'].isnull().sum()

C:\Users\sncap\AppData\Local\Temp\ipykernel_17248\1487776982.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['var_prod'] = (x['precio_actual'] / x['cantidad_actual']) * (x['cantidad_anterior'] / x['precio_anterior'])
C:\Users\sncap\AppData\Local\Temp\ipykernel_17248\385769678.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  variacion['var_prod'] = bol.apply(lambda row: 1 if row['nt_tipo'] in [351761, 351762] else row['var_prod'], axis=1)


In [18]:
#relativo de precios de los productos
relativo = (variacion.groupby(['region', 'cod_prod'])
            .agg(relativo = ('var_prod', lambda x: gmean(x)))
            .reset_index())

relativo

,region,cod_prod,relativo
0,1,0111101,NaN
1,1,0111201,NaN
2,1,0111202,0.983335
3,1,0111203,NaN
4,1,0111301,NaN
...,...,...,...
2258,8,1251104,1.000000
2259,8,1312009,NaN
2260,8,1312012,1.000000
2261,8,1321104,1.000000


In [19]:
# Step 1: Calculate geometric mean by grouping region and cod_prod
relativo = (bol01.groupby(['region', 'cod_prod'])
                  .agg(relativo=('var_prod', lambda x: gmean(x)))
                  .reset_index())

# Step 2: Left join with pon_prod DataFrame
relativo = (relativo.merge(pon_prod[['region', 'cod_prod', 'grupo_nombre', 'ponderacion_region', 'indice_anterior']], 
                           on=['region', 'cod_prod'], 
                           how='left')
                    .rename(columns={'relativo': 'geometric_mean'}))

# Step 3: Select required columns
relativo = relativo[['region', 'cod_prod', 'grupo_nombre', 'geometric_mean', 'ponderacion_region', 'indice_anterior']]

NameError: name 'bol01' is not defined